In [5]:
python setup.py install

SyntaxError: invalid syntax (1000827859.py, line 1)

In [6]:
import pandas as pd

import nltk
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

products = pd.read_csv('./dataset/products_no_china_no_token_v2.csv')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ModuleNotFoundError: No module named 'gensim'

In [ ]:
# 텍스트 데이터를 단어 단위로 토큰화
def string_to_list(input_string):
# 문자열을 rhdqor()를 기준으로 나누어 리스트로 변환
  return str(input_string).split(' ')


# string_to_list 함수를 사용하여 각 문장을 단어의 리스트로 변환
sentences = [string_to_list(sentence) for sentence in products['CATALOG_DESC']]

# Word2Vec 모델 초기화
model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, workers=4)
# 이 때, vector_size는 단어 벡터의 차원 수를 지정합니다.
# window는 주변 단어의 범위를 나타냅니다.
# min_count는 최소 등장 횟수를 의미합니다.

# 모델 학습
model.train(sentences, total_examples=len(sentences), epochs=10)

# 단어 벡터 확인
#word_vector = model.wv['document']
#print(word_vector)

In [ ]:
import re

keywords = pd.read_csv('/content/drive/MyDrive/REALFIANL/dataset/IPR리스트_231218.csv')

keywords['KEYWORD'] = keywords['KEYWORD'].apply(lambda x: re.sub('[^0-9a-zA-Z]', ' ', x))

keywords['KEYWORD']

In [ ]:
keyword_list = []

for word in keywords['KEYWORD']:
  keyword_list.append(word)

keyword_list[:4]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 두 단어의 코사인 유사도 계산 함수
def cosine_similarity(word1, word2):
    vec1 = model.wv[word1].reshape(1, -1)  # (1, 100)의 형태로 reshape
    vec2 = model.wv[word2]
    similarity = np.dot(vec1, vec2.T) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    return similarity[0, 0]  # 결과를 스칼라 값으로 얻기 위해 [0, 0]을 사용

# 예시: 'apetamin'과 다른 단어들 간의 유사도 계산
other_words_vectors = [model.wv[word] for word in sentences]


# 인덱스와 유사도를 데이터프레임에 추가
similarity = cosine_similarity(keyword_list, other_words_vectors)    
# products['SIMILARITY'] = similarity[0]

In [ ]:
import numpy as np

# 두 단어의 코사인 유사도 계산 함수
def cosine_similarity(word1, word2):
    vec1 = model.wv[word1].reshape(1, -1)  # (1, 100)의 형태로 reshape
    vec2 = model.wv[word2].reshape(1, -1)  # (1, 100)의 형태로 reshape
    similarity = np.dot(vec1, vec2.T) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    return similarity[0, 0]  # 결과를 스칼라 값으로 얻기 위해 [0, 0]을 사용

# 예시: 'apetamin'과 다른 단어들 간의 유사도 계산
other_words_vectors = [model.wv[word] for word in sentences]

# 유사도 계산
similarities = [cosine_similarity(keyword_list, word) for word in other_words_vectors]

# 데이터프레임에 추가
products['SIMILARITY'] = similarities